## Spanish Sentiment comparison


In [1]:
%load_ext autoreload
%autoreload 2

import random
import tweetnlp
import stanza
from tqdm.auto import tqdm
from pysentimiento import create_analyzer
from textblob import TextBlob
from datasets import load_dataset, ClassLabel

mteb = load_dataset("mteb/tweet_sentiment_multilingual", "spanish")
mteb = mteb.rename_column("text", "sentence")
mteb = mteb.cast_column("label", ClassLabel(3, names=["negative", "neutral", "positive"]))



amazon = load_dataset("SetFit/amazon_reviews_multi_es", split="test")

def convert_label(ex):
    if ex["label"] <= 1:
        return 0
    elif ex["label"] >= 3:
        return 2
    else:
        return 1

amazon = amazon.map(lambda ex: {"label": convert_label(ex)})
amazon = amazon.cast_column("label", ClassLabel(3, names=["negative", "neutral", "positive"]))
amazon = amazon.rename_column("text", "sentence")

# Es el mismo que MTEB
#tweet_nlp = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "spanish", split="test")
#
#tweet_nlp = tweet_nlp.rename_column("text", "sentence")


benchmark_datasets = [
    ("mteb", mteb["test"]),
    ("amazon", amazon),
]

#pysentimient + tweetnlp + stanza




def pysentimiento_analyzer(dataset):
    analyzer = create_analyzer("sentiment", lang="es")
    id2label = dataset.features["label"].names

    outs = analyzer.predict(dataset["sentence"])
    del analyzer
    if len(id2label) == 2:
        # Only positive/negative
        return ["negative" if x.probas["NEG"] > x.probas["POS"] else "positive" for x in outs]
    else:
        translation = {"NEU": "neutral", "POS": "positive", "NEG": "negative"}
        return [translation[x.output] for x in outs]

def stanza_analyzer(dataset):
    nlp = stanza.Pipeline(lang='es', processors='tokenize,sentiment', tokenize_no_ssplit=True)
    id2label = dataset.features["label"].names
    outs = nlp(dataset["sentence"])

    def _get_sentiment(x):
        if x.sentiment == 0:
            return "negative"
        elif x.sentiment == 2:
            return "positive"
        elif len(id2label) == 2:
            # Flip a coin
            if random.random() > 0.5:
                return "positive"
            else:
                return "negative"
        else:
            return "neutral"

    return [_get_sentiment(x) for x in outs.sentences]

def tweetnlp_analyzer(dataset):
    model = tweetnlp.load_model('sentiment')  # Or `model = tweetnlp.Sentiment()`
    id2label = dataset.features["label"].names
    outs = model.predict(dataset["sentence"])

    del model
    def get_tweetnlp_sentiment(x):
        if x["label"] in {"positive", "negative"}:
            return x["label"]
        elif len(id2label) == 2:
            # Flip a coin
            if random.random() > 0.5:
                return "positive"
            else:
                return "negative"
        else:
            return "neutral"

    return [get_tweetnlp_sentiment(x) for x in outs]

analyzers = {
    "pysentimiento": pysentimiento_analyzer,
    "tweetnlp": tweetnlp_analyzer,
    "stanza": stanza_analyzer,
    #"vader": vader_analyzer
}


2024-06-15 19:56:56,548	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-15 19:56:56,669	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Repo card metadata block was not found. Setting CardData to empty.


In [2]:
from sklearn.metrics import classification_report

results = []

for ds_name, dataset in tqdm(benchmark_datasets):
    preds = {}

    for k, analyzer in analyzers.items():
        print(k)
        preds[k] = analyzer(dataset)

    id2label = dataset.features["label"].names
    label2id = {v: k for k, v in enumerate(id2label)}


    for name, pred in tqdm(list(preds.items())):
        print(name)
        true_labels = dataset["label"]
        pred_labels = [label2id[x] for x in pred]

        ret = classification_report(true_labels, pred_labels, target_names=id2label, output_dict=True)

        res = {
            "Model": name,
            "Dataset": ds_name,
            "Macro F1": ret["macro avg"]["f1-score"],
            "Macro Precision": ret["macro avg"]["precision"],
            "Macro Recall": ret["macro avg"]["recall"],
        }

        results.append(res)


  0%|          | 0/2 [00:00<?, ?it/s]

pysentimiento


Map:   0%|          | 0/870 [00:00<?, ? examples/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 870
  Batch size = 32


tweetnlp


...

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5000
  Batch size = 32


tweetnlp


loading configuration file https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest/resolve/main/config.json from cache at /users/jmperez/.cache/huggingface/transformers/c26252806565e705085b65f69d7d544c05112fee06744845d6c067efcb278fff.31fdd4298ba667598119e493f82afb18fcd41f96366700ec7d6460c17c421feb
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment-latest",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "robe

OutOfMemoryError: CUDA out of memory. Tried to allocate 7.32 GiB. GPU 

In [ ]:
k

In [ ]:
import pandas as pd

df = pd.DataFrame(results)

(df.set_index(["Dataset", "Model"]) * 100).round(2)[["Macro Precision", "Macro Recall", "Macro F1"]]